In [1]:
import requests
import re
import os
from collections import Counter
import sys
import pprint
import json

### Задание №1

Доработать приложение по поиску авиабилетов, чтобы оно возвращало билеты по названию города, а не по IATA коду. (У aviasales есть для этого дополнительное API) Пункт отправления и пункт назначения должны передаваться в качестве параметров. Сделать форматированный вывод, который содержит в себе пункт отправления, пункт назначения, дату вылета, цену билета (можно добавить еще другие параметры по желанию)

Исходный код

In [2]:
req = requests.get("http://min-prices.aviasales.ru/calendar_preload?origin=LED&destination=NCE&one_way=true")

data = json.loads(req.text)
pprint.pprint(data)
print(data['best_prices'][0])

{'best_prices': [{'actual': True,
                  'depart_date': '2019-10-23',
                  'destination': 'NCE',
                  'distance': 2386,
                  'found_at': '2019-07-02T15:24:06.429243',
                  'gate': 'AMADEUS',
                  'number_of_changes': 1,
                  'origin': 'LED',
                  'return_date': None,
                  'show_to_affiliates': False,
                  'trip_class': 0,
                  'value': 6633.0},
                 {'actual': True,
                  'depart_date': '2019-10-24',
                  'destination': 'NCE',
                  'distance': 2386,
                  'found_at': '2019-07-01T17:26:51',
                  'gate': 'KLM/AirFrance',
                  'number_of_changes': 1,
                  'origin': 'LED',
                  'return_date': None,
                  'show_to_affiliates': False,
                  'trip_class': 0,
                  'value': 5764.0},
                 {'actual

                  'depart_date': '2019-09-14',
                  'destination': 'NCE',
                  'distance': 2386,
                  'found_at': '2019-06-25T15:33:10',
                  'gate': 'Trip.ru',
                  'number_of_changes': 1,
                  'origin': 'LED',
                  'return_date': None,
                  'show_to_affiliates': False,
                  'trip_class': 0,
                  'value': 6845.0},
                 {'actual': True,
                  'depart_date': '2019-12-19',
                  'destination': 'NCE',
                  'distance': 2386,
                  'found_at': '2019-07-02T15:24:06.429243',
                  'gate': 'AMADEUS',
                  'number_of_changes': 1,
                  'origin': 'LED',
                  'return_date': None,
                  'show_to_affiliates': False,
                  'trip_class': 0,
                  'value': 6936.0},
                 {'actual': True,
                  'depart_date'

                 {'actual': True,
                  'depart_date': '2019-10-05',
                  'destination': 'NCE',
                  'distance': 2386,
                  'found_at': '2019-07-02T15:24:06.429243',
                  'gate': 'AMADEUS',
                  'number_of_changes': 1,
                  'origin': 'LED',
                  'return_date': None,
                  'show_to_affiliates': False,
                  'trip_class': 0,
                  'value': 6936.0},
                 {'actual': True,
                  'depart_date': '2020-01-25',
                  'destination': 'NCE',
                  'distance': 2386,
                  'found_at': '2019-06-25T14:46:11',
                  'gate': 'Vueling',
                  'number_of_changes': 1,
                  'origin': 'LED',
                  'return_date': None,
                  'show_to_affiliates': False,
                  'trip_class': 0,
                  'value': 8090.0},
                 {'actual': Tru

Модернизированный код

In [3]:
origin = input('Введите город отправления: ')

Введите город отправления: Москва


In [4]:
destination = input('Введите город назначения: ')

Введите город назначения: Хабаровск


Отправляем запрос для получения IATA кодов городов отправления и назначения

In [5]:
req_iata = requests.get(f'https://www.travelpayouts.com/widgets_suggest_params?q=Из%20{origin}%20в%20{destination}')

In [6]:
data_iata = json.loads(req_iata.text)

destination_iata = data_iata['destination']['iata']
origin_iata = data_iata['origin']['iata']

In [7]:
print(f'IATA коды городов отправления {origin}: {origin_iata} и назначения {destination}: {destination_iata}')

IATA коды городов отправления Москва: MOW и назначения Хабаровск: KHV


Получаем данные об авиарейсе

In [8]:
req_avia = requests.get(f'http://min-prices.aviasales.ru/calendar_preload?origin={origin_iata}&destination={destination_iata}&one_way=true')

In [9]:
data_avia = json.loads(req_avia.text)

min_price = data_avia['best_prices'][len(data_avia['best_prices'])-1]['value']
depart_date = data_avia['best_prices'][len(data_avia['best_prices'])-1]['depart_date']

In [10]:
print(f'Минильная стоимость билета на рейс {origin} - {destination} {min_price} руб., дата вылета {depart_date}.')

Минильная стоимость билета на рейс Москва - Хабаровск 10116.0 руб., дата вылета 2019-09-03.


### Задание №2

В приложении парсинга википедии получить первую ссылку на другую страницу и вывести все значимые слова из неё. Результат записать в файл в форматированном виде

Исходный код

In [11]:
from requests import get
import re

def get_link(topic):
    link = "https://ru.wikipedia.org/wiki/" + topic.capitalize()
    return link

def get_topic_page(topic):
    link = get_link(topic)
    html = get(link).text
    return html

def get_topic_text(topic):
    html_content = get_topic_page(topic)
    words = re.findall("[а-яА-Я]+", html_content)
    return words

def get_common_words(topic):
    words_list = get_topic_text(topic)
    rate={}
    for word in words_list:
        if word in rate:
            rate[word]+=1
        else:
            rate[word]=1
    rate_list = list(rate.items())
    rate_list.sort(key = lambda x :-x[1])

    return rate_list

def visualize_common_words(topic):
    words = get_common_words(topic)
    for w in words[0:10]:
        print(f'{w[0]} встречается {w[1]} раз')

list1 = visualize_common_words('Дерево')
print(list1)

в встречается 155 раз
и встречается 141 раз
на встречается 82 раз
с встречается 73 раз
деревья встречается 52 раз
или встречается 47 раз
Дерево встречается 44 раз
до встречается 42 раз
деревьев встречается 40 раз
раздел встречается 40 раз
None


Модернизированный код

In [12]:
def get_link(topic):
    if topic.startswith("http"):
        return topic
    else:
        return "https://ru.wikipedia.org/wiki/" + topic.capitalize()

def get_topic_page(topic):
    link = get_link(topic)
    html = requests.get(link).text
    return html

def get_topic_text(topic):
    content = get_topic_page(topic)
    words = re.findall("[а-яА-Я]+", content)
    return words

Перепишем функцию get_common_words с помощью счетчика 

In [13]:
def get_common_words(topic):
    cnt = Counter(" ".join(get_topic_text(topic)).lower().split(" "))
    return cnt.most_common()

In [14]:
def get_additional_links(topic):
    page = get_topic_page(topic)
    page_text = re.findall('id=\"Ссылки\">[\s\S]*', page)
    if len(page_text) > 0:
        page_text = page_text[0]
        page_text = re.sub('</ul>[\s\S]*', "", page_text)
        additional_links = re.findall('\"http[^\">]*\">', page_text)
        for n in range(len(additional_links)):
            additional_links[n] = re.sub("^[^ht]*", "", additional_links[n])
            additional_links[n] = re.sub("[^\w]*$", "", additional_links[n])
        return sorted(additional_links)
    else:
        return []

In [15]:
def recording_to_file(most_common, filepath):
    with open(filepath, "wt", encoding="utf-8") as output:
        for common in most_common:
            output.write("".join([str(common[0]), "\t", str(common[1]), "\n"]))

In [16]:
topic = input('Введите искомую фразу для поиска: ')

Введите искомую фразу для поиска: футбол


In [17]:
os.makedirs('output', exist_ok=True)

recording_to_file(get_common_words(topic), os.path.join('output', topic + ".txt"))

additional_links = get_additional_links(topic)
print(f"Всего дополнительных ссылок: {len(additional_links)}")
print(*additional_links, sep="\n")

for link in additional_links:
    common_words = get_common_words(link)
    output_path = os.path.join('output', re.sub("\W", "", link) + ".txt")
    recording_to_file(common_words, output_path)

Всего дополнительных ссылок: 4
http://conmebol.com
http://ru.uefa.com
https://commons.wikimedia.org/wiki/Category:Association_football?uselang=ru
https://www.fifa.com
